In [35]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use("bmh")
plt.rcParams['figure.figsize'] = 10, 6 # It’s a dictionary of most matplotlib styling that you set at the start of your notebook and it will apply to all your plots.It’s a dictionary of most matplotlib styling that you set at the start of your notebook and it will apply to all your plots.


import pandas as pd
import numpy as np
import scipy as sc
from scipy import signal

import matplotlib.pyplot as plt
import h5py
import obspy
from obspy.signal.filter import envelope
from obspy.clients.fdsn import Client
from tqdm import tqdm #tqdm is a Python library that allows you to output a smart progress bar 
from glob import glob #is a function that’s used to search for files that match a specific file pattern or name.
import tsfel #Time Series Feature Extraction Library is a Python package for feature extraction on time series data.
import random

from scipy import signal


client = Client('IRIS')

import os
import pickle #Pickling” is the process whereby a Python object hierarchy is converted into a byte stream

import matplotlib.pyplot as plt
import numpy as np

from scatseisnet import ScatteringNetwork
import matplotlib.dates as mdates

from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import KMeans #The k-means clustering method is an unsupervised machine learning technique used to identify clusters of data objects in a dataset

#%config InlineBackend.figure_format = "png"

#from Feature_Extraction import compute_hibert

import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [36]:
def extract_waveforms(cat, file_name):
    
    st = []
    cat_trace = cat['trace_name'].values
    for i in range(len(cat_trace)):

            
 
        f = h5py.File(file_name, 'r')
            
            
        bucket = cat['trace_name'].values[i].split('$')[0]
        # ind means index
        ind = int(cat['trace_name'].values[i].split('$')[1].split(',')[0])
        # ENZ
        st.append(f['/data/'+bucket][ind, :3, :18000])
        
    return st


def convert_into_traces(stream, cat):
    channels = ['BHE','BHN','BHZ']
    
    st = []
    for i in range(len(stream)):
        for j in range(len(stream[i])):
            tr = obspy.Trace(stream[i][j])
            tr.stats.channel = channels[j]
            tr.stats.station = cat['station_code'].values[i]
            tr.stats.network = cat['station_network_code'].values[i]
            tr.stats.sampling_rate = 100
            tr.stats.starttime = obspy.UTCDateTime(cat['trace_start_time'].values[i])
            
            st.append(tr)
            
    return obspy.Stream(st)

def plot_waveforms(tr_exotic_Z, title = 'Thunder Waveforms', xlim = [10,100]):



    # Create a figure with subplots
    fig, axs = plt.subplots(len(tr_exotic_Z), 2, figsize=(12, 12))

    # Plot time series and spectrograms
    for i in range(len(tr_exotic_Z)):
        time = tr_exotic_Z[i].times()
        # Plot time series in left subplot
        axs[i, 0].plot(time, tr_exotic_Z[i].data)
        axs[i, 0].set_xlabel('Time')
        axs[i, 0].set_ylabel('Amplitude')
        axs[i, 0].set_title(tr_exotic_Z[i].id)
        axs[i,0].set_xlim(xlim[0], xlim[1])

        # Compute and plot spectrogram in right subplot
        f, t_spec, Sxx = signal.spectrogram(tr_exotic_Z[i].data, fs = 100)
        axs[i, 1].imshow(Sxx, aspect='auto', cmap='jet', origin='lower', extent=[t_spec.min(), t_spec.max(), f.min(), f.max()])
        axs[i, 1].set_xlabel('Time')
        axs[i, 1].set_ylabel('Frequency')
        axs[i, 1].set_title(str(tr_exotic_Z[i].stats.starttime))
        axs[i, 1].set_xlim(xlim[0], xlim[1])
        axs[i, 1].set_ylim(0, 30)
        #axs[i, 1].set_yscale('log')

    # Labelling the figure. 
    fig.suptitle(title, fontsize = 20)


    # Adjust the spacing between subplots
    plt.tight_layout()

    # Show the plot
    plt.show()

In [37]:
# collecting waveform data and corresponding catalog.
exotic_file_name = "/data/whd01/yiyu_data/PNWML/exotic_waveforms.hdf5"
exotic_csv_file = "/data/whd01/yiyu_data/PNWML/exotic_metadata.csv"

noise_file_name = "/data/whd01/yiyu_data/PNWML/noise_waveforms.hdf5"
noise_csv_file = "/data/whd01/yiyu_data/PNWML/noise_metadata.csv"

comcat_file_name = "/data/whd01/yiyu_data/PNWML/comcat_waveforms.hdf5"
comcat_csv_file = "/data/whd01/yiyu_data/PNWML/comcat_metadata.csv"

In [38]:
## collecting the catalog and storing it. 
# pd.read_csv: Read a comma-separated values (csv) file into DataFrame (tables).
cat_exotic = pd.read_csv(exotic_csv_file)
cat_noise = pd.read_csv(noise_csv_file)
cat_comcat = pd.read_csv(comcat_csv_file)

In [49]:
cat_exotic

,event_id,source_type,station_network_code,station_channel_code,station_code,station_location_code,station_latitude_deg,station_longitude_deg,station_elevation_m,trace_name,trace_sampling_rate_hz,trace_start_time,trace_S_arrival_sample,trace_S_onset,trace_P_arrival_sample,trace_P_onset,trace_snr_db,snr_final
0,pnsn3166678,thunder,CC,BH,CPCO,--,43.722,-121.233,2135.0,"bucket3$0,:3,:18001",100,2021-08-02T03:17:19.000000Z,NaN,NaN,7000.0,emergent,23.922|19.759|28.482,28.482
1,pnsn3166673,thunder,UW,EH,NCO,--,43.704,-121.140,1908.0,"bucket3$1,:3,:18001",100,2021-08-02T03:06:44.000000Z,NaN,NaN,7000.0,emergent,nan|nan|18.054,18.054
2,pnsn3165628,thunder,CC,BH,CPCO,--,43.722,-121.233,2135.0,"bucket3$2,:3,:18001",100,2021-07-26T12:37:35.000000Z,NaN,NaN,7000.0,impulsive,13.149|20.561|19.350,19.350
3,pnsn3165633,thunder,CC,BH,CPCO,--,43.722,-121.233,2135.0,"bucket3$3,:3,:18001",100,2021-07-26T12:30:25.000000Z,NaN,NaN,7000.0,emergent,6.389|6.103|3.934,3.934
4,pnsn3162793,thunder,CC,BH,TMBU,--,43.602,-121.145,1743.0,"bucket3$4,:3,:18001",100,2021-06-24T15:01:08.000000Z,NaN,NaN,7000.0,emergent,2.768|0.671|1.021,1.021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9262,pnsn830593,surface event,UW,EH,EDM,--,46.197,-122.151,1609.0,"bucket1$8993,:3,:18001",100,2002-01-02T17:38:01.000000Z,NaN,NaN,7000.0,impulsive,nan|nan|-1.162,-1.162
9263,pnsn830593,surface event,UW,EH,YEL,--,46.209,-122.189,1750.0,"bucket1$8994,:3,:18001",100,2002-01-02T17:37:59.000000Z,NaN,NaN,7000.0,impulsive,nan|nan|14.469,14.469
9264,pnsn830593,surface event,UW,EH,SEP,--,46.200,-122.191,2116.0,"bucket1$8995,:3,:18001",100,2002-01-02T17:37:59.000000Z,NaN,NaN,7000.0,impulsive,nan|nan|4.575,4.575
9265,pnsn829903,surface event,UW,EH,YEL,--,46.209,-122.189,1750.0,"bucket1$8996,:3,:18001",100,2002-01-01T21:34:30.000000Z,NaN,NaN,7000.0,impulsive,nan|nan|12.728,12.728


In [50]:
cat_noise

,source_type,station_network_code,station_channel_code,station_code,station_location_code,station_latitude_deg,station_longitude_deg,station_elevation_m,trace_name,trace_sampling_rate_hz,trace_start_time
0,noise,UW,BH,TTW,--,47.694,-121.690,542.0,"bucket1$0,:3,:15001",100,2002-09-21T00:50:20.960000Z
1,noise,UW,HH,ERW,--,48.454,-122.626,387.0,"bucket2$0,:3,:15001",100,2002-09-21T00:50:20.960000Z
2,noise,UW,BH,GNW,--,47.564,-122.825,220.0,"bucket3$0,:3,:15001",100,2002-09-21T00:50:20.960000Z
3,noise,UW,BH,OPC,--,48.100,-123.413,90.0,"bucket4$0,:3,:15001",100,2002-09-21T00:50:20.960000Z
4,noise,UW,BH,GNW,--,47.564,-122.825,220.0,"bucket5$0,:3,:15001",100,2002-09-13T14:54:54.050000Z
...,...,...,...,...,...,...,...,...,...,...,...
51457,noise,UW,HH,KTSAP,--,47.518,-122.939,166.9,"bucket1$5122,:3,:15001",100,2022-12-27T10:43:24.810000Z
51458,noise,UW,HH,TKEY,--,47.179,-122.771,47.4,"bucket2$5119,:3,:15001",100,2022-12-27T10:43:24.810000Z
51459,noise,UW,HH,SKOKO,--,47.409,-123.367,949.6,"bucket3$5116,:3,:15001",100,2022-12-27T10:43:24.810000Z
51460,noise,UW,HH,WINDI,--,47.314,-123.332,594.0,"bucket4$5113,:3,:15001",100,2022-12-27T10:43:24.810000Z


In [51]:
cat_comcat

,event_id,source_origin_time,source_latitude_deg,source_longitude_deg,source_type,source_depth_km,preferred_source_magnitude,preferred_source_magnitude_type,preferred_source_magnitude_uncertainty,source_depth_uncertainty_km,...,trace_snr_db,source_type_pnsn_label,source_local_magnitude,source_local_magnitude_uncertainty,source_duration_magnitude,source_duration_magnitude_uncertainty,source_hand_magnitude,trace_missing_channel,trace_has_offset,snr_final
0,uw10564613,2002-10-03T01:56:49.530000Z,48.553,-122.520,earthquake,14.907,2.10,md,0.030,1.68,...,6.135|3.065|11.766,eq,NaN,NaN,2.10,0.030,NaN,0,1,11.766
1,uw10564613,2002-10-03T01:56:49.530000Z,48.553,-122.520,earthquake,14.907,2.10,md,0.030,1.68,...,nan|nan|22.583,eq,NaN,NaN,2.10,0.030,NaN,2,0,22.583
2,uw10564613,2002-10-03T01:56:49.530000Z,48.553,-122.520,earthquake,14.907,2.10,md,0.030,1.68,...,1.756|3.057|3.551,eq,NaN,NaN,2.10,0.030,NaN,0,1,3.551
3,uw10564613,2002-10-03T01:56:49.530000Z,48.553,-122.520,earthquake,14.907,2.10,md,0.030,1.68,...,nan|nan|27.185,eq,NaN,NaN,2.10,0.030,NaN,2,0,27.185
4,uw10568748,2002-09-26T07:00:04.860000Z,48.481,-123.133,earthquake,22.748,2.90,md,0.030,0.91,...,10.881|17.107|2.242,eq,NaN,NaN,2.90,0.030,NaN,0,1,2.242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183904,uw61802032,2022-01-15T01:30:38.770000Z,48.617,-123.032,earthquake,12.410,0.26,md,0.416,0.74,...,17.389|11.516|12.029,eq,NaN,NaN,0.26,0.416,NaN,0,0,12.029
183905,uw61802032,2022-01-15T01:30:38.770000Z,48.617,-123.032,earthquake,12.410,0.26,md,0.416,0.74,...,9.564|8.578|2.713,eq,NaN,NaN,0.26,0.416,NaN,0,0,2.713
183906,uw61802032,2022-01-15T01:30:38.770000Z,48.617,-123.032,earthquake,12.410,0.26,md,0.416,0.74,...,6.539|-0.560|-0.724,eq,NaN,NaN,0.26,0.416,NaN,0,0,-0.724
183907,uw61802032,2022-01-15T01:30:38.770000Z,48.617,-123.032,earthquake,12.410,0.26,md,0.416,0.74,...,3.382|1.918|2.539,eq,NaN,NaN,0.26,0.416,NaN,0,0,2.539


In [40]:
## making SNR in correct format
snrs_exotic = []
for i in range(len(cat_exotic['trace_snr_db'])):
    snrs_exotic.append(np.array(cat_exotic['trace_snr_db'].values[i].split('|')).astype('float')[-1])
    
snrs_comcat = []
for i in range(len(cat_comcat['trace_snr_db'])):
    snrs_comcat.append(np.array(cat_comcat['trace_snr_db'].values[i].split('|')).astype('float')[-1])

In [41]:
cat_exotic['snr_final'] = snrs_exotic
cat_comcat['snr_final'] = snrs_comcat

In [42]:
cat_thunder = cat_exotic[cat_exotic['source_type'] == 'thunder']
cat_surface = cat_exotic[cat_exotic['source_type'] == 'surface event']
cat_sonic = cat_exotic[cat_exotic['source_type'] == 'sonic boom']
cat_plane = cat_exotic[cat_exotic['source_type'] == 'plane crash']

cat_explosion = cat_comcat[cat_comcat['source_type'] == 'explosion']
cat_earthquake = cat_comcat[cat_comcat['source_type'] == 'earthquake']

In [43]:
cat_thunder_good = cat_thunder.sort_values(by = 'snr_final', ascending = False).iloc[0:5]
cat_surface_good = cat_surface.sort_values(by = 'snr_final', ascending = False).iloc[0:5]
cat_sonic_good = cat_sonic.sort_values(by = 'snr_final', ascending = False).iloc[0:5]
cat_plane_good = cat_plane.sort_values(by = 'snr_final', ascending = False).iloc[0:5]
cat_explosion_good = cat_explosion.sort_values(by = 'snr_final', ascending = False).iloc[0:5]
cat_earthquake_good = cat_earthquake.sort_values(by = 'snr_final', ascending = False).iloc[0:5]

In [44]:
cat_thunder_good

,event_id,source_type,station_network_code,station_channel_code,station_code,station_location_code,station_latitude_deg,station_longitude_deg,station_elevation_m,trace_name,trace_sampling_rate_hz,trace_start_time,trace_S_arrival_sample,trace_S_onset,trace_P_arrival_sample,trace_P_onset,trace_snr_db,snr_final
9,pnsn3138083,thunder,CC,BH,KWBU,--,43.752,-121.312,1783.0,"bucket3$9,:3,:18001",100,2020-08-06T04:36:48.000000Z,NaN,NaN,7000.0,emergent,20.705|22.407|43.313,43.313
82,pnsn2169988,thunder,CC,EH,KWBU,--,43.752,-121.312,1783.0,"bucket3$82,:3,:18001",100,2015-07-10T22:37:25.000000Z,NaN,NaN,7000.0,impulsive,37.146|36.494|37.051,37.051
57,pnsn3096073,thunder,CC,BH,CPCO,--,43.722,-121.233,2135.0,"bucket3$57,:3,:18001",100,2019-05-30T18:31:39.000000Z,NaN,NaN,7000.0,emergent,35.483|37.782|37.009,37.009
89,pnsn2150153,thunder,CC,BH,CPCO,--,43.722,-121.233,2135.0,"bucket3$89,:3,:18001",100,2015-06-04T17:37:38.000000Z,7500.0,emergent,7000.0,impulsive,31.590|35.392|34.393,34.393
28,pnsn3104343,thunder,CC,BH,JRO,--,46.275,-122.218,1219.0,"bucket3$28,:3,:18001",100,2019-08-10T06:39:02.000000Z,NaN,NaN,7000.0,impulsive,21.601|23.785|31.992,31.992


In [45]:
stream_exotic = extract_waveforms(cat_thunder_good, exotic_file_name)
tr_thunder_Z = convert_into_traces(stream_exotic, cat_thunder_good).select(channel = '*HZ')

stream_earthquake = extract_waveforms(cat_earthquake_good, comcat_file_name)
tr_earthquake_Z = convert_into_traces(stream_earthquake, cat_earthquake_good).select(channel = '*HZ')


stream_surface = extract_waveforms(cat_surface_good, exotic_file_name)
tr_surface_Z = convert_into_traces(stream_surface, cat_surface_good).select(channel = '*HZ')


stream_explosion = extract_waveforms(cat_explosion_good, comcat_file_name)
tr_explosion_Z = convert_into_traces(stream_explosion, cat_explosion_good).select(channel = '*HZ')

stream_sonic = extract_waveforms(cat_sonic_good, exotic_file_name)
tr_sonic_Z = convert_into_traces(stream_sonic, cat_sonic_good).select(channel = '*HZ')

In [46]:
st

NameError: name 'st' is not defined

In [47]:
stream_exotic

[array([[ -548.42170411,  -317.43219471,  -221.92250365, ...,
         -1623.64258145, -1595.32619475, -1475.39529843],
        [ 1933.05822706,  2092.75504798,  2153.54961527, ...,
          1235.63798493,  1225.19508843,  1283.54836934],
        [-2337.25011165, -2321.41604802, -2308.2488137 , ...,
         -2352.52046078, -2365.00545705, -2371.56170917]]),
 array([[  9.75072601,  10.75044253,   8.00011294, ...,   0.75002727,
          -1.75021262,  -3.2506728 ],
        [119.74981953, 121.25001946, 122.75031474, ..., 128.75031036,
         124.74999862, 121.99994094],
        [126.75054395, 125.75050915, 118.99957822, ..., 128.50049181,
         126.75032883, 122.24968848]]),
 array([[ 249.50012676,  205.61517418,  171.73462168, ...,  271.17719222,
          292.00067332,  309.63452593],
        [-742.99590817, -742.20906719, -746.24825664, ..., -769.40922039,
         -761.7466941 , -758.78237737],
        [2549.00745233, 2559.73014089, 2557.50672409, ..., 2499.23484656,
         2